In [ ]:
import os
from diffsky.experimental.disk_bulge_modeling.generate_bulge_disk_sample import (
    get_bulge_disk_test_sample,
    get_bulge_disk_decomposition,
)
from diffsky.experimental.disk_bulge_modeling.disk_bulge_kernels import (
    get_observed_quantity_pop,
)

from diffaux.validation.plot_utilities import (
    get_zindexes,
)

from diffaux.size_modeling.fit_size_data import (
    generate_sizes,
    get_median_sizes,
    Samples_zFit,
    get_color_mask,
    median_size_vs_z,
    _sigmoid,
)

from jax import random as jran
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict, namedtuple
from itertools import zip_longest

In [ ]:
ran_key = jran.key(0)
halo_key, ran_key = jran.split(ran_key, 2)
lgmp_min = 11.0
redshift = 0.05
Lbox = 75.0
diffstar_cens = get_bulge_disk_test_sample(halo_key, lgmp_min=lgmp_min, redshift=redshift, Lbox=Lbox)
print(list(diffstar_cens.keys()))

In [ ]:
zvalues = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5]
redshifts = diffstar_cens["z_table"]
zindexes, zs = get_zindexes(zvalues, redshifts)

In [ ]:
disk_bulge_key, ran_key = jran.split(ran_key, 2)
diffstar_cens = get_bulge_disk_decomposition(disk_bulge_key, diffstar_cens)
print(diffstar_cens.keys())

In [ ]:
# get observed disk and bulge masses
tarr = diffstar_cens["t_table"]
tobs = jnp.full(len(diffstar_cens["logsm_obs"]), diffstar_cens["t_obs"])
zobs = jnp.full(len(diffstar_cens["logsm_obs"]), redshift)
bulge_sm_obs = get_observed_quantity_pop(tobs, tarr, diffstar_cens["smh_bulge"])
disk_sm_obs = np.power(10, diffstar_cens["logsm_obs"]) - bulge_sm_obs
print(Samples_zFit)
print(np.min(np.log10(bulge_sm_obs)), np.max(np.log10(bulge_sm_obs)))
print(np.min(np.log10(disk_sm_obs)), np.max(np.log10(disk_sm_obs)))

In [ ]:
# set up redshift and fake color arrays
zobs = jnp.full(len(diffstar_cens["logsm_obs"]), redshift)
disk_color = np.random.uniform(low=-0.2, high=1.45, size=len(diffstar_cens["logsm_obs"]))
bulge_color = np.random.uniform(low=1.55, high=2.5, size=len(diffstar_cens["logsm_obs"]))

In [ ]:
from diffaux.size_modeling.fit_size_data import read_fit_parameters, zFitParameters

fit_pars, fits = read_fit_parameters(zFitParameters)
print(fit_pars)

In [ ]:
# disk sizes
Re = {}
R_med = {}
labels = ["Disk", "Bulge"]
masses = [disk_sm_obs, bulge_sm_obs]
colors = [disk_color, bulge_color]
for l, sample, mass, color in zip(labels, Samples_zFit, masses, colors):
    Re[l], R_med[l] = generate_sizes(
        fit_pars,
        jnp.log10(mass),
        zobs,
        color,
        samples=[sample],
        fit_type="Re_vs_z",
    )
    print(l)
    maske = np.isfinite(Re[l])
    maskm = np.isfinite(R_med[l])
    print(np.count_nonzero(maske), np.count_nonzero(maske))
    print(np.min(R_med[l]), np.max(R_med[l]))
    print(np.min(Re[l]), np.max(Re[l]))
# bulge sizes

In [ ]:
from importlib.resources import files

SIZE_DATA_DIR = files("diffaux").joinpath("data/GalaxySizes/zhang_2019")
fn = "Zhang_et_al_2019_Size_Morphology.txt"
data = np.genfromtxt(
    os.path.join(SIZE_DATA_DIR, fn),
    skip_header=2,
    dtype=[("type", "U12"), ("alpha", float), ("beta", float), ("gamma", float), ("M0", float)],
    names=True,
)
print(data)
keys = [str(data[k][0]) for k in range(len(data))]
zhang = dict(zip(keys, [[float(data[k][l]) for l in range(1, len(data[k]))] for k in range(len(data))]))
print(zhang["late"])


def zhang_et_al_fit(M, alpha, beta, gamma, M0, h=0.678):
    # correct M to units of Msun/h**2
    M = M / (h * h)
    return gamma * (M / M0) ** alpha * (1 + M / M0) ** (beta - alpha)


Mtest = np.logspace(7.5, 11.5, 8)
print(Mtest)
for k, v in zhang.items():
    print(k, zhang_et_al_fit(Mtest, *v))

In [ ]:
plotdir = "/Users/kovacs/cosmology/BulgeDisk/DiskBulgePlots/DiskBulgeSizes"
from diffaux.validation.plot_utilities import get_nrow_ncol, save_fig
from scipy.stats import binned_statistic

lMbins = np.linspace(7.5, 11.5, 17)
nrow = 1
ncol = 1
fit_colors = ("yellow", "c", "coral", "dodgerblue", "orange", "cornflowerblue")
fig, ax = plt.subplots(nrow, ncol, figsize=(ncol * 7, nrow * 5))
for l, mass, c, cpts in zip(labels, masses, ("blue", "r"), ("royalblue", "tomato")):
    logM = np.log10(mass)
    lM, _, _ = binned_statistic(logM, logM, statistic="mean", bins=lMbins)
    mask = np.isfinite(lM)
    y, _, _ = binned_statistic(logM, Re[l], statistic="mean", bins=lMbins)
    std, _, _ = binned_statistic(logM, Re[l], statistic="std", bins=lMbins)
    ax.scatter(logM, Re[l], alpha=0.3, color=cpts, label="{}".format(l))
    ax.plot(lM[mask], y[mask], label="{} Median Size".format(l), color=c)
    ax.fill_between(lM, y - std, y + std, color=c, alpha=0.4)

for (k, v), f in zip(zhang.items(), fit_colors):
    if "B/T" not in k:
        ax.plot(np.log10(Mtest), zhang_et_al_fit(Mtest, *v), label="{} Zhang (2019)".format(k), color=f)


ax.set_xlabel("$\\log_{10}(M^*/M_\\odot)$")
ax.set_ylabel("$R_e$ (kpc)")
ax.set_yscale("log")
ax.set_ylim(0.5, 14.0)
ax.legend(loc="best")


fig.suptitle("Generated Sizes z = {:.2f}".format(redshift), y=0.92, fontweight="bold")
pltname = "Disk_Bulge_Re_vs_Mstar_z_{:.2f}.png".format(redshift)
save_fig(fig, plotdir, pltname)

In [ ]:
# Now assign sizes to whole galaxy and plot based on B/T cut
# galaxy sizes
B_over_T_obs = get_observed_quantity_pop(tobs, tarr, diffstar_cens["bth"])
bmask = B_over_T_obs <= 0.5
logM_BT = [diffstar_cens["logsm_obs"][bmask], diffstar_cens["logsm_obs"][~bmask]]
# assign color based on B/T
color_BT = np.zeros(len(diffstar_cens["logsm_obs"]))
color_BT[bmask] = disk_color[bmask]
color_BT[~bmask] = bulge_color[~bmask]

Re_BT, R_med_BT = generate_sizes(
    fit_pars,
    diffstar_cens["logsm_obs"],
    zobs,
    color_BT,
    samples=Samples_zFit,
    fit_type="Re_vs_z",
)

In [ ]:
# plot
logM_BT = [diffstar_cens["logsm_obs"][bmask], diffstar_cens["logsm_obs"][~bmask]]
lMbins = np.linspace(7.5, 11.5, 17)
nrow = 1
ncol = 1
fit_colors = ("darkorange", "c")
fig, ax = plt.subplots(nrow, ncol, figsize=(ncol * 7, nrow * 5))
for sample, logM, c, cpts in zip(Samples_zFit, logM_BT, ("blue", "r"), ("royalblue", "tomato")):
    lM, _, _ = binned_statistic(logM, logM, statistic="mean", bins=lMbins)
    mask = np.isfinite(lM)
    y, _, _ = binned_statistic(logM, Re[l], statistic="mean", bins=lMbins)
    std, _, _ = binned_statistic(logM, Re[l], statistic="std", bins=lMbins)
    ax.scatter(logM, Re[l], alpha=0.3, color=cpts, label="{}".format(l))
    ax.plot(lM[mask], y[mask], label="{} Median Size".format(l), color=c)
    ax.fill_between(lM, y - std, y + std, color=c, alpha=0.4)

for (k, v), f in zip(zhang.items(), fit_colors):
    if "B/T" in k:
        ax.plot(np.log10(Mtest), zhang_et_al_fit(Mtest, *v), label="{} Zhang (2018)".format(k), color=f)


ax.set_xlabel("$\\log_{10}(M^*/M_\\odot)$")
ax.set_ylabel("$R_e$ (kpc)")
ax.set_ylim(0.5, 14.0)
ax.legend(loc="best")


fig.suptitle("Generated Sizes z = {:.2f}".format(redshift), y=0.92, fontweight="bold")
pltname = "Disk_Bulge_Re_vs_Mstar_z_{:.2f}.png".format(redshift)
save_fig(fig, plotdir, pltname)